In [11]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Input, Embedding, Dense, Dot, Activation, Lambda
from tensorflow.keras.models import Model

In [12]:


value_vocab = set() #set of all leaf values
path_vocab = set() #set of all distinct paths
tags_vocab = set() #set of all distinct function tags

#vocab sizes and embedding dimensions
value_vocab_size = len(value_vocab)
path_vocab_size = len(path_vocab)
tags_vocab_size = len(tags_vocab)
embedding_dim = 128  # As per your image


# Inputs
input_value1 = Input(shape=(None,), name='value1_input')
input_path = Input(shape=(None,), name='path_input')
input_value2 = Input(shape=(None,), name='value2_input')

# Embedding Layers for the value and path vocabularies
value_embedding = Embedding(input_dim=value_vocab_size, output_dim=embedding_dim, name='value_embedding')
path_embedding = Embedding(input_dim=path_vocab_size, output_dim=embedding_dim, name='path_embedding')

# Embed inputs
embedded_value1 = value_embedding(input_value1)
embedded_path = path_embedding(input_path)
embedded_value2 = value_embedding(input_value2)

# Concatenate all embeddings
embedded_concat = tf.concat([embedded_value1, embedded_path, embedded_value2], axis=1)

# Dense transformation with tanh for context vector transformation
transformed_vector = Dense(embedding_dim, activation='tanh')(embedded_concat)

# Attention mechanism
attention_weights = Dense(1, activation='softmax')(transformed_vector)
weighted_context = Dot(axes=1)([attention_weights, transformed_vector])

# Fully connected layer and softmax for output prediction
output = Dense(len(tags_vocab), activation='softmax')(weighted_context)

# Define and compile the model
model = Model(inputs=[input_value1, input_path, input_value2], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
value1_input (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
path_input (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
value2_input (InputLayer)       [(None, None)]       0                                            
__________________________________________________________________________________________________
value_embedding (Embedding)     (None, None, 128)    256         value1_input[0][0]               
                                                                 value2_input[0][0]           